In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (col, when)

import os
import findspark
findspark.init()

# --- ADD THIS BLOCK ---
# Allow the Security Manager for Java 21+
os.environ["JDK_JAVA_OPTIONS"] = "-Djava.security.manager=allow"
# ----------------------

spark = SparkSession.builder \
    .master("local[*]") \
    .getOrCreate()

# Initialize Spark
spark = SparkSession.builder.appName("Traffy").getOrCreate()

#Turn off ANSI mode to allow silent null coercion
spark.conf.set("spark.sql.ansi.enabled", "false")

NOTE: Picked up JDK_JAVA_OPTIONS: -Djava.security.manager=allow
NOTE: Picked up JDK_JAVA_OPTIONS: -Djava.security.manager=allow
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/06 20:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/06 20:40:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# ---------------------------------------------------------
# LOAD DATA (Bronze Layer)
# ---------------------------------------------------------

df_air = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .csv("../../data/raw/bangkok_air_raw.csv")

In [3]:
# ---------------------------------------------------------
# Cleaning and Feature Engineering
# ---------------------------------------------------------

df_air_clean = df_air \
    .withColumn("pm25_category", 
        when(col("avg_pm25_ug_m3") <= 25, "Very Good")
        .when(col("avg_pm25_ug_m3") <= 37, "Good")
        .when(col("avg_pm25_ug_m3") <= 50, "Moderate")
        .when(col("avg_pm25_ug_m3") <= 90, "Unhealthy for Sensitive")
        .otherwise("Unhealthy")
    )

In [4]:
# ---------------------------------------------------------
# EXPORT TO CSV (Silver Layer)
# ---------------------------------------------------------

output_file = "../../data/cleansed/bangkok_air_clean.csv"

print(f"💾 Converting to Pandas and saving to: {output_file} ...")

try:
    pdf_final = df_air_clean.toPandas()

    folder_path = os.path.dirname(output_file)
    if folder_path and not os.path.exists(folder_path):
        os.makedirs(folder_path)

    pdf_final.to_csv(
        output_file, 
        index=False, 
        encoding='utf-8-sig',
        header=True
    )

    print("✅ Export completed successfully.")

except Exception as e:
    print(f"❌ Error: {e}")

💾 Converting to Pandas and saving to: ../../data/cleansed/bangkok_air_clean.csv ...
✅ Export completed successfully.
